In [1]:
import os
import requests
import pandas as pd
from tqdm import tqdm

Steps: https://disc.gsfc.nasa.gov/information/howto?keywords=python&title=How%20to%20Access%20GES%20DISC%20Data%20Using%20Python

In [2]:
output_directory = "files"
os.makedirs(output_directory, exist_ok=True)

In [3]:
df = pd.read_csv(
    "/Users/kyla/Downloads/modis_mod13a2_v61_664cedd8f73d8165.csv", encoding="cp1252"
).reset_index()
print(len(df))
df.head(1)

646


,index,Local Granule ID,Entity ID,Acquisition Start Date,Acquisition End Date,Horizontal Tile Number,Vertical Tile Number,Day/Night Indicator,Version Number,Product Generation Algorithm,...,Center Longitude dec,NW Corner Lat dec,NW Corner Long dec,NE Corner Lat dec,NE Corner Long dec,SE Corner Lat dec,SE Corner Long dec,SW Corner Lat dec,SW Corner Long dec,Display ID
0,MOD13A2.A2009353.h11v04.061.2021149103412.hdf,MOD13A2.A2009353.h11v04.061,2009/12/19,2010/01/03,11,4,Both,61,6.1.1,Passed,...,49.9863,-109.0855,50.0921,-93.3968,39.8489,-78.1497,39.7728,-91.3388,MOD13A2.A2009353.h11v04.061,2458890598


In [4]:
df.values[0][2], df.values[0][0]

('2009/12/19', 'MOD13A2.A2009353.h11v04.061.2021149103412.hdf')

In [5]:
modis_kind = df.values[0][0].split(".")[0]
modis_kind

'MOD13A2'

In [6]:
errored_files = []
for row in tqdm(df.values):
    url = f"https://e4ftl01.cr.usgs.gov/DP131/MOLT/{modis_kind}.061/{row[2].replace('/','.')}/{row[0]}"
    result = requests.get(url)
    path_out = os.path.join(output_directory, row[0])
    try:
        result.raise_for_status()
        f = open(path_out, "wb")
        f.write(result.content)
        f.close()
    except Exception as e:
        print(f"{row[0]} -- {e} -- error code: {result.status_code}")
        errored_files.append(url)

100%|██████████| 646/646 [1:01:17<00:00,  5.69s/it]
